### Galois Fields of Degree 2

Burton Rosenberg

November 4, 2021



In [174]:

class QuadInt:
    
    def __init__(self,p,d):
        assert d%p!=0
        assert not self.qr(d,p)

        # p should be a prime but we only hope so
        assert p>2

        self.p = p
        self.d = d%p

    def qr(self,a,p):
        """
        check if a is quadradic residue mod p
        """
        return a**((p-1)//2)%p==1
        
    def get_mod(self):
        return self.p
    
    def get_des(self):
        return self.d

    def canonical(self,a):
        return (a[0]%self.p,a[1]%self.p)
    
    def zero_p(self,a):
        a = self.canonical(a)
        return a[0]==0 and a[1]==0
    
    def one_p(self,a):
        a = self.canonical(a)
        return a[0]==1 and a[1]==0
    
    def equal_p(self,a,b):
        a = self.canonical(a)
        b = self.canonical(b)
        return a[0]==b[0] and a[1]==b[1]

    def zero(self):
        return (0,0)
    
    def one(self):
        return (1,0)

    def conj(self,a):
        return (a[0],-a[1]%self.p)

    def minus(self,a):
        return (-a[0]%self.p,-a[1]%self.p)

    def add(self,a,b):
        c = (a[0]+b[0])%self.p
        d = (a[1]+b[1])%self.p
        return (c,d)
    
    def mult(self,a,b):
        c = (a[0]*b[0]+self.d*a[1]*b[1]) %self.p
        d = (a[0]*b[1]+a[1]*b[0]) %self.p
        return (c,d)
    
    def exp(self,a,n):
        if n==0:
            return (1,0)
        if n==1:
            return a
        if n%2==0:
            c = self.exp(a,n//2)
            return self.mult(c,c)
        c = self.exp(a,n-1)
        return self.mult(a,c)
 
    def extended_gcd(self,a,b):
        """
        extended GCD algorithm. recursive.
        returns (d,s,t) where d = s*a+t*b 
        and d = gcd(a,b)
        """
        assert(
            a>=0 and b>=0 )
        if b==0:
            return (a,1,0)
        (q,r) = divmod(a,b)
        (d,s,t) = self.extended_gcd(b,r)
        # gcd(a, b) == gcd(b, r) == s*b + t*r == s*b + t*(a - q*b)
        return (d,t,s-q*t)

    def invert_base(self,c):
        assert c[1]==0
        a = c[0]
        (d,t,s) = self.extended_gcd(a,self.p)
        assert 1==d
        return (t%self.p,0)
    
    def invert(self,a):
        if self.zero_p(a):
            return (0,0)

        c = (a[0],-a[1])
        d = self.mult(a,c)
        d_inv = self.invert_base(d)
        return self.mult(c,d_inv)
        
    def next(self,a):
        if a[1]==(self.p-1):
            return ((a[0]+1)%self.p,0)
        return (a[0],(a[1]+1)%self.p)
        
    def __str__(self):
        return f'Z/{self.p}Z[rad({self.d})]'


In [175]:
qi = QuadInt(11,7)
print(qi)

def test_inv(qi):
    p = qi.get_mod()
    a = qi.one()
    while not qi.zero_p(a):
        b = qi.invert(a)
        c = qi.mult(a,b)
        print(f'{a} inverse is {b}')
        assert qi.one_p(c), f'{a}'
        a = qi.next(a)

test_inv(qi)

Z/11Z[rad(7)]
(1, 0) inverse is (1, 0)
(1, 1) inverse is (9, 2)
(1, 2) inverse is (2, 7)
(1, 3) inverse is (3, 2)
(1, 4) inverse is (10, 4)
(1, 5) inverse is (6, 3)
(1, 6) inverse is (6, 8)
(1, 7) inverse is (10, 7)
(1, 8) inverse is (3, 9)
(1, 9) inverse is (2, 4)
(1, 10) inverse is (9, 9)
(2, 0) inverse is (6, 0)
(2, 1) inverse is (3, 4)
(2, 2) inverse is (10, 1)
(2, 3) inverse is (5, 9)
(2, 4) inverse is (1, 9)
(2, 5) inverse is (7, 10)
(2, 6) inverse is (7, 1)
(2, 7) inverse is (1, 2)
(2, 8) inverse is (5, 2)
(2, 9) inverse is (10, 10)
(2, 10) inverse is (3, 7)
(3, 0) inverse is (4, 0)
(3, 1) inverse is (7, 5)
(3, 2) inverse is (1, 3)
(3, 3) inverse is (3, 8)
(3, 4) inverse is (2, 1)
(3, 5) inverse is (8, 5)
(3, 6) inverse is (8, 6)
(3, 7) inverse is (2, 10)
(3, 8) inverse is (3, 3)
(3, 9) inverse is (1, 8)
(3, 10) inverse is (7, 6)
(4, 0) inverse is (3, 0)
(4, 1) inverse is (9, 6)
(4, 2) inverse is (7, 2)
(4, 3) inverse is (6, 1)
(4, 4) inverse is (5, 6)
(4, 5) inverse is (8, 1)
(

In [176]:
for i in range(25):
    print(qi.exp((0,2),i))

(1, 0)
(0, 2)
(6, 0)
(0, 1)
(3, 0)
(0, 6)
(7, 0)
(0, 3)
(9, 0)
(0, 7)
(10, 0)
(0, 9)
(5, 0)
(0, 10)
(8, 0)
(0, 5)
(4, 0)
(0, 8)
(2, 0)
(0, 4)
(1, 0)
(0, 2)
(6, 0)
(0, 1)
(3, 0)


In [177]:
p = qi.get_mod()
a = qi.one()
while not qi.zero_p(a):
    b1 = qi.mult(a,a)
    a_minus = qi.minus(a)
    b2 = qi.mult(a_minus,a_minus)
    assert qi.equal_p(b1,b2)
    print(f'{a} or {a_minus} squared is {b1}')
    a = qi.next(a)
        

(1, 0) or (10, 0) squared is (1, 0)
(1, 1) or (10, 10) squared is (8, 2)
(1, 2) or (10, 9) squared is (7, 4)
(1, 3) or (10, 8) squared is (9, 6)
(1, 4) or (10, 7) squared is (3, 8)
(1, 5) or (10, 6) squared is (0, 10)
(1, 6) or (10, 5) squared is (0, 1)
(1, 7) or (10, 4) squared is (3, 3)
(1, 8) or (10, 3) squared is (9, 5)
(1, 9) or (10, 2) squared is (7, 7)
(1, 10) or (10, 1) squared is (8, 9)
(2, 0) or (9, 0) squared is (4, 0)
(2, 1) or (9, 10) squared is (0, 4)
(2, 2) or (9, 9) squared is (10, 8)
(2, 3) or (9, 8) squared is (1, 1)
(2, 4) or (9, 7) squared is (6, 5)
(2, 5) or (9, 6) squared is (3, 9)
(2, 6) or (9, 5) squared is (3, 2)
(2, 7) or (9, 4) squared is (6, 6)
(2, 8) or (9, 3) squared is (1, 10)
(2, 9) or (9, 2) squared is (10, 3)
(2, 10) or (9, 1) squared is (0, 7)
(3, 0) or (8, 0) squared is (9, 0)
(3, 1) or (8, 10) squared is (5, 6)
(3, 2) or (8, 9) squared is (4, 1)
(3, 3) or (8, 8) squared is (6, 7)
(3, 4) or (8, 7) squared is (0, 2)
(3, 5) or (8, 6) squared is (8, 8)
